In [ ]:
# Reinforcement Learning for Stock Trading with Live Trading and Backtesting
# This script integrates Alpaca API for live trading and uses Backtesting.py for historical analysis

# Install required libraries before running:
# pip install alpaca-trade-api numpy pandas bt

import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
import bt  # Backtesting.py alternative

# Define Alpaca API keys (Replace with actual credentials)
API_KEY = "your_api_key_here"
SECRET_KEY = "your_secret_key_here"
BASE_URL = "https://paper-api.alpaca.markets"

# Initialize Alpaca API connection
api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version="v2")

# Function to fetch historical stock data from Alpaca
def get_stock_data(symbol, timeframe="1Day", limit=365):
    """
    Fetches historical stock data from Alpaca using the updated get_bars() method.

    Parameters:
    - symbol: Stock ticker (e.g., 'AAPL')
    - timeframe: Bar timeframe (1Min, 5Min, 1H, 1Day)
    - limit: Number of historical bars to fetch

    Returns:
    - DataFrame with historical price data
    """
    barset = api.get_bars(symbol, timeframe=timeframe, limit=limit).df
    return barset

# Function to place trades using Alpaca API
def place_trade(symbol, qty, side="buy"):
    """
    Places a trade on Alpaca.
    
    Parameters:
    - symbol: Stock ticker (e.g., 'AAPL')
    - qty: Number of shares
    - side: "buy" or "sell"
    
    Returns:
    - Order confirmation
    """
    order = api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type="market",
        time_in_force="gtc"
    )
    return order

# Simulated RL-based trading strategy (random for now, replace with a trained model)
def rl_trading_strategy(state):
    """
    Reinforcement learning model placeholder.
    Returns: 0 (hold), 1 (buy), 2 (sell)
    """
    return np.random.choice([0, 1, 2])

# Function to execute RL-based live trading
def trade_with_rl(symbol, shares=10):
    """
    Uses RL model to make live trades on Alpaca.
    """
    df = get_stock_data(symbol, timeframe="1Day", limit=10)
    state = np.array([df["close"].values[-1]])  # Using last close price as state
    action = rl_trading_strategy(state)
    if action == 1:
        print(f"Buying {shares} shares of {symbol}")
        place_trade(symbol, qty=shares, side="buy")
    elif action == 2:
        print(f"Selling {shares} shares of {symbol}")
        place_trade(symbol, qty=shares, side="sell")
    else:
        print(f"Holding {symbol}")

# Backtesting RL strategy using Backtesting.py
class RLTradingStrategy(bt.Strategy):
    def __init__(self):
        self.signal = np.random.choice([0, 1, -1], size=len(self.data))  # Placeholder for RL model decisions
    
    def next(self):
        if self.signal[self.data.index.get_loc(self.data.index[self.bar])] == 1:
            self.buy()
        elif self.signal[self.data.index.get_loc(self.data.index[self.bar])] == -1:
            self.sell()

# Fetch historical stock data for backtesting
df = get_stock_data("AAPL", timeframe="1Day", limit=365)

# Convert price data to Backtesting.py format
data = bt.Data(df)

# Create and run backtest
bt_strategy = RLTradingStrategy()
backtest = bt.Backtest(bt_strategy, data, initial_cash=10000, commission=0.001)
result = bt.run(backtest)

# Print Backtest Results
print("Final Portfolio Value:", result.get_value())
print("Total Return (%):", result.get_return() * 100)
print("Max Drawdown (%):", result.get_max_drawdown() * 100)
print("Sharpe Ratio:", result.get_sharpe_ratio())

# Run Live Trading (Comment out if only running backtesting)
# trade_with_rl("AAPL", shares=10)
